## 1 Problem Statement

In the 2nd quarter of 2019, a new research from Juniper Research (Research, Forecasting & Consultancy for Digital Technology Markets) found the number of successful retail chatbot interactions will reach 22 billion by 2023, up from an estimated 2.6 billion in 2019. 
(https://www.juniperresearch.com/press/press-releases/chatbot-interactions-retail-reach-22-billion-2023) 

Chatbot can been seen as a strong potential to reduce operational cost when comparing the values in terms of manhours cost and having the system running 24/7. 

Another benefits of using chatbot through the support and questions from customers, insights on where are the pain points can also be derive. These feedbacks can then be used to help the company improve or focus more to help improve user experience and increase user stickiness.

5 airlines (AirAsia, AmericanAir, British Airway, Delta and SouthWest Air) decided to come together to do a joint venture project of building a Chatbot to help reduce opertional cost and increased their sales revenue. The project will be completed in 4 stages. 

- 1st stage: Chatbot able to deal with feedback from the passengers.
- 2nd stage: Chatbot able to predict which feedback belongs to the respective airline.
- 2nd stage: Chatbot handle general queries from passengers.
- 3rd stage: Chatbot able to query the database for details like flight booking, cancellation etc.

With that, being a data scientist at AirAsia, I was task to complete the first stage of the project to do a test run to see the general response from the passengers and making predictions if possible.


## 2 Using Kaggle Jupyter Notebook

The dataset was taken from https://www.kaggle.com/thoughtvector/customer-support-on-twitter .
It consist more than 2.8 millions rows.

Loading, exploring, and processing of the big dataset in the intial stage was done using kaggle jupyter where they can provide higher RAM and GPU as compared to my local machine. Since the dataset is sitting in their server, loading and reading is also naturally much faster when using their jupypter.

Once exploring was done, i started to break the dataset into smaller set so i can used it for my multiclass classfication and topic modelling using my local notebook at a later stage.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


All libraries used

In [1]:
import requests
import time
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sns.set(style='whitegrid', palette='muted')

## 3 Data Import

In [2]:
#twcs_df = pd.read_csv("/kaggle/input/customer-support-on-twitter/twcs/twcs.csv")
twcs_df = pd.read_csv("../dataset/twcs.csv")

In [3]:
twcs_df.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


In [4]:
twcs_df.shape

(2811774, 7)

In [5]:
author = pd.DataFrame(twcs_df['author_id'].value_counts())

In [6]:
author.head(50)

,author_id
AmazonHelp,169840
AppleSupport,106860
Uber_Support,56270
SpotifyCares,43265
Delta,42253
Tesco,38573
AmericanAir,36764
TMobileHelp,34317
comcastcares,33031
British_Airways,29361


## 4 Query/Respond

The start of a new tweet from a customer: 
- "in_response_to_tweet_id" column = "Nan"
- "inbound" column = True

In [7]:
start = twcs_df[pd.isnull(twcs_df["in_response_to_tweet_id"]) & twcs_df["inbound"]]
reply = pd.merge(start, twcs_df, left_on='tweet_id', right_on='in_response_to_tweet_id')

In [8]:
# Outgoing replies (from companies)
reply = reply[reply['inbound_y'] ^ True]
print(f'Data shape: {reply.shape}')
reply.head()

Data shape: (794299, 14)


,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,17,NaN,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",19,NaN,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,NaN,20.0


In [9]:
#output the predicted value to a new .csv file 
#reply.to_csv('mycsvfile.csv',index=False)
reply.to_csv('../dataset/mycsvfile.csv',index=False)

In [10]:
reply.drop(columns = ['tweet_id_x','inbound_x','created_at_x','response_tweet_id_x','in_response_to_tweet_id_x',\
                      'tweet_id_y','inbound_y','created_at_y','response_tweet_id_y','in_response_to_tweet_id_y'],\
          inplace = True)

In [11]:
#Dataframe contains with needed columns
reply.head()

,author_id_x,text_x,author_id_y,text_y
0,115712,@sprintcare is the worst customer service,sprintcare,@115712 Can you please send us a private messa...
1,115712,@sprintcare is the worst customer service,sprintcare,@115712 I would love the chance to review the ...
2,115712,@sprintcare is the worst customer service,sprintcare,@115712 Hello! We never like our customers to ...
3,115713,@115714 y’all lie about your “great” connectio...,sprintcare,@115713 H there! We'd definitely like to work ...
4,115715,"@115714 whenever I contact customer support, t...",sprintcare,@115715 Please send me a private message so th...


In [12]:
reply.shape

(794299, 4)

## 5 Breaking into smaller dataset

In [13]:
#Splitting dataset into smaller set.
#Dataset 1: Delta
Delta_df =  reply[reply['author_id_y'] == 'Delta']
#Delta_df.to_csv('Delta_df.csv',index=False)
Delta_df.to_csv('../dataset/Delta_df.csv',index=False)

In [14]:
#Splitting dataset into smaller set.
#Dataset 2: AmericanAir
AmericanAir_df =  reply[reply['author_id_y'] == 'AmericanAir']
#AmericanAir_df.to_csv('AmericanAir_df.csv',index=False)
AmericanAir_df.to_csv('../dataset/AmericanAir_df.csv',index=False)

In [15]:
#Splitting dataset into smaller set.
#Dataset 3: British_Airways
British_Airways_df =  reply[reply['author_id_y'] == 'British_Airways']
#British_Airways_df.to_csv('British_Airways_df.csv',index=False)
British_Airways_df.to_csv('../dataset/British_Airways_df.csv',index=False)

In [16]:
#Splitting dataset into smaller set.
#Dataset 4: SouthwestAir
SouthwestAir_df =  reply[reply['author_id_y'] == 'SouthwestAir']
#SouthwestAir_df.to_csv('SouthwestAir_df.csv',index=False)
SouthwestAir_df.to_csv('../dataset/SouthwestAir_df.csv',index=False)

In [17]:
#Splitting dataset into smaller set.
#Dataset 5: AirAsiaSupport
AirAsiaSupport_df =  reply[reply['author_id_y'] == 'AirAsiaSupport']
#AirAsiaSupport_df.to_csv('AirAsiaSupport_df.csv',index=False)
AirAsiaSupport_df.to_csv('../dataset/AirAsiaSupport_df.csv',index=False)